In [1]:
%matplotlib notebook

import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import animation
import IPython
import IPython.display
import IPython.core.display
from os.path import join, exists
from os import makedirs

output_dir = '../../outf/HW2-4'
if not exists(output_dir):
    makedirs(output_dir)

In [2]:
def make_animation(plotx, E, xl=(0, 1), yl=(0, 1), inter=25, display=False):
    '''
    takes a graph and motion of vertexes in 2D and returns an animation
    E: list of edges (each edge is a pair of vertexes)
    plotx: a matrix of states ordered as (x1, y1, x2, y2, ..., xn, yn) in the rows and time in columns
    xl and yl define the display boundaries of the graph
    inter is the interval between each point in ms
    '''
    fig = mpl.figure.Figure()
    mpl.backends.backend_agg.FigureCanvasAgg(fig)
    ax = fig.add_subplot(111, autoscale_on=False, xlim=xl, ylim=yl)
    ax.grid()

    list_of_lines = []
    for i in E: #add as many lines as there are edges
        line, = ax.plot([], [], 'o-', lw=2)
        list_of_lines.append(line)

    def animate(i):
        for e in range(len(E)):
            vx1 = plotx[2*(E[e][0]-1),i]
            vy1 = plotx[2*(E[e][0]-1)+1,i]
            vx2 = plotx[2*(E[e][1]-1),i]
            vy2 = plotx[2*(E[e][1]-1)+1,i]
            list_of_lines[e].set_data([vx1,vx2],[vy1,vy2])
        return list_of_lines

    def init():
        return animate(0)

    ani = animation.FuncAnimation(fig, animate, np.arange(0, len(plotx[0,:])),
        interval=inter, blit=True, init_func=init)
    plt.close(fig)
    plt.close(ani._fig)
    if(display==True):
        IPython.display.display_html(IPython.core.display.HTML(ani.to_html5_video()))
    return ani

In [3]:
class SecondOrderControl:
    def __init__(self, G, poses, zref, time=10, dt=0.01, k=0.5):
        self.G = G
        self.zref = zref
        self.time = time
        self.dt = dt
        self.k = k
        self.N = G.number_of_nodes()

        self.T = int(time / dt)
        self.u = np.zeros((self.T, self.N, 2))
        self.x = np.zeros((self.T, self.N, 2))
        self.x[0] = poses

        self.L = nx.laplacian_matrix(G).toarray()
        self.D = nx.incidence_matrix(G, oriented=True).toarray()

    def draw_initial_conditions(self):
        pos = {}
        plt.figure()
        for i, position in enumerate(self.x[0]):
            pos[i+1] = position
        pos_shadow = {key: (value[0] + 0.005, value[1] + 0.005) for key, value in pos.items()}
        nx.draw(self.G, pos_shadow, with_labels=False, node_size=600, node_color='none', edgecolors='gray', edge_color='gray', width=3.0, alpha=0.5)

        nx.draw(self.G, pos, with_labels=True, node_size=600, edge_color='red', width=2.0, linewidths=3.0, edgecolors='black', node_color='white', font_size=12, font_weight='bold')
        plt.title('Initial Conditions')
        plt.show()

    def run_simulation(self):
        for t in range(1, self.T):
            self.u[t - 1] = -self.k * self.L @ self.x[t - 1] + self.k * self.D @ self.zref
            self.x[t] = self.x[t - 1] + self.u[t - 1] * self.dt

    def plot_results(self, filename=None, display=True):
        E = list(self.G.edges())
        X = np.zeros((2 * self.N, self.T))
        for i in range(self.T):
            for j in range(self.N):
                X[2 * j, i] = self.x[i, j, 0]
                X[2 * j + 1, i] = self.x[i, j, 1]

        ani = make_animation(X, E, xl=(0, 5), yl=(0, 5), inter=10, display=display)
        if filename:
            writervideo = animation.FFMpegWriter(fps=20, codec='h264', extra_args=['-pix_fmt', 'yuv420p'])
            ani.save(filename, writer=writervideo)

        return ani

## Incomplete Graph

In [9]:
N = 4
G = nx.Graph()
G.add_nodes_from([1, 2, 3, 4])
G.add_edges_from([(1, 2), (2, 3), (1, 3), (1, 4), (3, 4)])
a=7/2-np.sqrt(7/8)
poses=np.array([[3,4],[4,4],[4,3],[a,a]])
zref = np.array([[1, 1],[1, 1], [np.sqrt(2), np.sqrt(2)], [1.5, 1.5],  [1.5, 1.5]])

control = SecondOrderControl(G, poses,zref)
control.draw_initial_conditions()

<IPython.core.display.Javascript object>

## Simulation under Random Initial Condition

In [6]:
poses_random=np.random.uniform(0,5,(N, 2))
control = SecondOrderControl(G, poses_random,zref)
control.run_simulation()
control.plot_results(filename=join(output_dir,'simulations_animation.gif'))

Your browser does not support the video tag.

## Complete graph

In [10]:
N = 4
G_new = nx.Graph()
G_new.add_nodes_from([1, 2, 3, 4])
G_new.add_edges_from([(1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)])
b=np.sqrt(7/4)+np.sqrt(1/2)
zref_new = np.array([[1, 1], [np.sqrt(2), np.sqrt(2)], [1.5, 1.5], [1, 1], [b, b], [1.5, 1.5]])
a=7/2-np.sqrt(7/8)
poses=np.array([[3,4],[4,4],[4,3],[a,a]])
control_new = SecondOrderControl(G_new,poses, zref_new)
control_new.draw_initial_conditions()

<IPython.core.display.Javascript object>

# Simulation under Random Initial Condition

In [7]:
control_new = SecondOrderControl(G_new,poses_random, zref_new)
control_new.run_simulation()
control_new.plot_results(filename=join(output_dir,'simulations_animation_complete.gif'))

Your browser does not support the video tag.